## AirBnB Optimal Price Predictor

This notebook downloads the data from remote used in this project and then cleans it. Ready to be used to build a model for predictions.


In [1]:
# Imports
import pandas as pd
from sklearn.model_selection import train_test_split
import category_encoders as ce
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras import backend as K
from sklearn.pipeline import make_pipeline
import numpy as np

In [2]:
import sklearn
import tensorflow
print('Pandas version: ',pd.__version__)
print('sklearn version: ',sklearn.__version__)
print('category_encoders version: ',ce.__version__)
print('tensorflow version: ',tensorflow.__version__)
print('numpy version: ',np.__version__)

# pipenv installation string
#  pip install pandas==1.1.5 scikit-learn==0.22.2 category_encoders==2.2.2 tensorflow==2.5.0

"""
Output from above should match the following in order to run the following code successfully:-

Pandas version:  1.1.5
sklearn version:  0.22.2
category_encoders version:  2.2.2
tensorflow version:  2.5.0
numpy version:  1.18.5
"""

Pandas version:  1.1.5
sklearn version:  0.22.2
category_encoders version:  2.2.2
tensorflow version:  2.5.0
numpy version:  1.18.5


'\nOutput from above should match the following in order to run the following code successfully:-\n\nPandas version:  1.1.5\nsklearn version:  0.22.2\ncategory_encoders version:  2.2.2\ntensorflow version:  2.5.0\nnumpy version:  1.18.5\n'

In [3]:
df = pd.read_csv('../data/raw/australia_visualisations_listings.csv.csv')

df

FileNotFoundError: [Errno 2] No such file or directory: '../data/raw/australia_visualisations_listings.csv.csv'

In [17]:
# Check out for any null values
df.isnull().sum()
# No null values

id                                    0
name                                 20
host_id                               0
host_name                            78
neighbourhood_group                   0
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
number_of_reviews_ltm                 0
last_review                       25878
reviews_per_month                 25878
calculated_host_listings_count        0
availability_365                      0
dtype: int64

In [18]:
# Drop unnecessary columns and
# Rearrange columns in X, y format
df = df[['latitude', 'longitude', 'neighbourhood_group', 'room_type',
       'minimum_nights', 'number_of_reviews', 'calculated_host_listings_count',
       'availability_365', 'price']]

In [19]:
df.describe(include='all')

,latitude,longitude,neighbourhood_group,room_type,minimum_nights,number_of_reviews,calculated_host_listings_count,availability_365,price
count,153914.000000,153914.000000,153914,153914,153914.000000,153914.000000,153914.000000,153914.000000,153914.000000
unique,NaN,NaN,9,4,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,New South Wales,Entire home/apt,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,63530,114630,NaN,NaN,NaN,NaN,NaN
mean,-33.257543,146.183371,NaN,NaN,3.925848,29.800902,15.511221,164.495264,255.932709
std,5.074834,9.664016,NaN,NaN,21.398849,53.670211,51.409165,140.028747,509.293129
min,-43.580050,96.831310,NaN,NaN,1.000000,0.000000,1.000000,0.000000,0.000000
25%,-37.572468,144.965470,NaN,NaN,1.000000,1.000000,1.000000,1.000000,99.000000
50%,-33.886870,150.378895,NaN,NaN,2.000000,8.000000,2.000000,155.000000,163.000000
75%,-31.884408,151.287350,NaN,NaN,3.000000,34.000000,5.000000,313.000000,286.000000


In [20]:
# Split in X and y

X = df[['latitude', 'longitude', 'neighbourhood_group', 'room_type',
       'minimum_nights', 'number_of_reviews', 'calculated_host_listings_count',
       'availability_365']]
y = df[['price']]

X.shape, y.shape

((153914, 8), (153914, 1))

In [21]:

# Not splitting in order to see if the OHE transformation works without split
#
# # Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.15, shuffle=True, )
#
# # See the shape
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((130826, 8), (23088, 8), (130826, 1), (23088, 1))

In [22]:
# TODO: Investigate why this is not working
# Investigating
# OneHot Encode data
def ohe_transform(X_train, X_test):
       """
       OneHotEncoder transformer for X_train, X_test data categorical information
       :param X_train:
       :param X_test:
       :return: X_train_ohe, X_test_ohe"""
       ohe = ce.OrdinalEncoder()
       ohe.fit(X_train)
       X_train_ohe = ohe.transform(X_train)
       X_test_ohe = ohe.transform(X_test)

       return X_train_ohe, X_test_ohe

X_train_ohe, X_test_ohe = ohe_transform(X_train, X_test)
X_train.shape, X_test.shape
# X_test_ohe = ohe_transform(X)

# Doing the same thing as above therefore ignoring for now
#
# # prepare input data in quantitative data
# def transform_categorical_data(X_train, X_test):
#        oe = OrdinalEncoder()
#        oe.fit(X_train)
#        X_train_oe = oe.transform(X_train)
#        X_test_oe = oe.transform(X_test)
#        return X_train_oe, X_test_oe
#
# # Run the function to have encoded X_train and X_test
# X_train, X_test = transform_categorical_data(X_train, X_test)

((130826, 8), (23088, 8))

In [9]:
"""
def transform_categorical_data(X_train, X_test):
       oe = OrdinalEncoder()
       oe.fit(X_train)
       X_train_oe = oe.transform(X_train)
       X_test_oe = oe.transform(X_test)
       return X_train_oe, X_test_oe

# Run the function to have encoded X_train and X_test
X_train_oe, X_test_oe = transform_categorical_data(X_train, X_test)"""

'\ndef transform_categorical_data(X_train, X_test):\n       oe = OrdinalEncoder()\n       oe.fit(X_train)\n       X_train_oe = oe.transform(X_train)\n       X_test_oe = oe.transform(X_test)\n       return X_train_oe, X_test_oe\n\n# Run the function to have encoded X_train and X_test\nX_train_oe, X_test_oe = transform_categorical_data(X_train, X_test)'

In [9]:
# The code above didn't work therefore using pipeline to do OneHotEncoding

# Define basic pipeline
# pipeline = make_pipeline(
#     ce.OneHotEncoder(use_cat_names=True)
# )
#
# #fit on train, score on val
# pipeline.fit(X_train, y_train)
# # print('Val accuracy', pipeline.score(X_val, y_val))
#
# #before encoding
# X_train.shape
#
# #after encoding
# encoder = pipeline.named_steps['onehotencoder']
# X_train_ohe = encoder.transform(X_train)
# #
# X_train_ohe.head()

NameError: name 'X_train' is not defined

In [11]:
# Transform X_test using OHE
# X_test_ohe = encoder.transform(X_test)

In [23]:
# Export DataFrame data to numpy values
X_train_np = X_train_ohe.values
X_test_np = X_test_ohe.values

In [13]:
# Code below cannot be used to save model due to the presence of custom model
"""
# Defining metric coeff_determination to get R-squared value for linear regression metric for target y
def coeff_determination(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true-y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

# Defining vanilla Feedforward neural network
model = Sequential()
model.add(Dense(10, input_dim=X_train_np.shape[1], activation='relu'))
model.add(Dense(128, activation='relu'))

model.add(Dense(1, activation='linear'))

# model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['accuracy' ,'mean_absolute_error'])
model.compile(loss='mean_absolute_error', optimizer='adam', metrics=[coeff_determination])

model.summary()"""


"\n# Defining metric coeff_determination to get R-squared value for linear regression metric for target y\ndef coeff_determination(y_true, y_pred):\n    SS_res =  K.sum(K.square( y_true-y_pred ))\n    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )\n    return ( 1 - SS_res/(SS_tot + K.epsilon()) )\n\n# Defining vanilla Feedforward neural network\nmodel = Sequential()\nmodel.add(Dense(10, input_dim=X_train_np.shape[1], activation='relu'))\nmodel.add(Dense(128, activation='relu'))\n\nmodel.add(Dense(1, activation='linear'))\n\n# model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['accuracy' ,'mean_absolute_error'])\nmodel.compile(loss='mean_absolute_error', optimizer='adam', metrics=[coeff_determination])\n\nmodel.summary()"

In [24]:
# Defining vanilla Feedforward neural network
model = Sequential()
model.add(Dense(10, input_dim=X_train_np.shape[1], activation='relu'))
model.add(Dense(128, activation='relu'))

model.add(Dense(1))

# model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['accuracy' ,'mean_absolute_error'])
model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 10)                90        
_________________________________________________________________
dense_4 (Dense)              (None, 128)               1408      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 129       
Total params: 1,627
Trainable params: 1,627
Non-trainable params: 0
_________________________________________________________________


In [26]:
# Fit the model
model.fit(X_train_np, y_train, batch_size=32,epochs=4, verbose=1, validation_data=(X_test_np, y_test))

Epoch 1/20
4089/4089 [==============================] - 2s 603us/step - loss: 137.4245 - mean_absolute_error: 137.4245 - val_loss: 140.6792 - val_mean_absolute_error: 140.6792

### The losses seems high and coeff_determination is very low. Not a good model

In [27]:
predicted_prices = model.predict(X_test_ohe)
predicted_prices

array([[224.09467],
       [163.36339],
       [189.09036],
       ...,
       [172.58675],
       [181.93542],
       [204.55878]], dtype=float32)

In [19]:
# Actual
y_test

,price
0,190
1,1000
2,68
3,166
4,290
...,...
153909,135
153910,195
153911,68
153912,40


In [16]:
predicted_prices = model.predict(X_test_ohe)
predicted_prices

NameError: name 'X_test_ohe' is not defined

In [28]:
# Actual
y_test

,price
104563,750
37231,106
95178,383
105091,99
105096,169
...,...
101361,883
126107,50
88415,280
69227,180


In [19]:
# Checkout a prediction from the model
test_prediction = {'latitude': -38.25482, 'longitude': 144.50328, 'room_type_Entire home/apt': 1,
       'room_type_Private room': 0, 'room_type_Shared room': 0,
       'room_type_Hotel room': 0, 'minimum_nights': 2, 'number_of_reviews': 3,
       'calculated_host_listings_count': 1, 'availability_365': 31}
test_prediction_df = pd.DataFrame([test_prediction])

model.predict(test_prediction_df)

array([[193.05498]], dtype=float32)

In [20]:
# Save Model
model.save('../models/airbnbpredict.h5')

In [21]:
# Test by loading and predicting

# load model
test_model = load_model('../models/airbnbpredict.h5')

# Model Summary
test_model.summary()

# Checkout a prediction from the model
test_prediction = {'latitude': -38.25482, 'longitude': 144.50328, 'room_type_Entire home/apt': 1,
       'room_type_Private room': 0, 'room_type_Shared room': 0,
       'room_type_Hotel room': 0, 'minimum_nights': 2, 'number_of_reviews': 3,
       'calculated_host_listings_count': 1, 'availability_365': 31}
test_prediction_df = pd.DataFrame([test_prediction])

test_model.predict(test_prediction_df)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                110       
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1408      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 1,647
Trainable params: 1,647
Non-trainable params: 0
_________________________________________________________________


array([[193.05498]], dtype=float32)